R environment

In [ ]:
library(Seurat)
library(dplyr)
library(cowplot)
library(ggplot2)
library(copykat)
library(patchwork)
library(future)
source("modified_copykat.R")
plan("multiprocess", workers = 12)
options(future.globals.maxSize = 10000 * 1024^10,
        future.rng.onMisuse = 'ignore')
sessionInfo()

Attaching SeuratObject


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘patchwork’


The following object is masked from ‘package:cowplot’:

    align_plots




R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /hpc/pmc_stunnenberg/cruiz/miniconda3/envs/r_pHGG_project/lib/libopenblasp-r0.3.12.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] future_1.21.0      patchwork_1.1.1    copykat_1.0.4      ggplot2_3.3.3     
[5] cowplot_1.1.1      dplyr_1.0.6        SeuratObject_4.0.1 Seurat_4.0.1.9005 

loaded via a namespace (and not attached):
  [1] nlme_3.1-152          spatstat.sparse_2.0-0 matrixStats_0.58.0   
  [

### Extract metadata to be added on the trained model

In [ ]:
cnv_1 <- readRDS('data/copykat_ref_normal_cells_part_I.rds')
cnv_2 <- readRDS('data/copykat_ref_normal_cells_part_II.rds')

In [ ]:
our.names <- grep('NH', names(cnv_2), value = TRUE)
our.names

[1] "NH16-2366" "NH17-1245" "NH17-1258" "NH17-161"  "NH17-1953" "NH17-2069"
 [7] "NH17-2680" "NH17-442"  "NH18-1406" "NH19-123"  "NH19-565"

In [ ]:
our <- cnv_2[our.names]
our

$`NH16-2366`
An object of class Seurat 
36498 features across 1872 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$`NH17-1245`
An object of class Seurat 
36498 features across 880 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$`NH17-1258`
An object of class Seurat 
36498 features across 3521 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$`NH17-161`
An object of class Seurat 
36498 features across 4357 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$`NH17-1953`
An object of class Seurat 
36498 features across 3400 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$`NH17-2069`
An object of class Seurat 
36498 features across 3509 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$`NH17-2680`
An object of class Seurat 
36498 features across 3549 samples within 1 assay 
Active assay: RNA (3649

In [ ]:
# extract the information from our own GB cohort
cnv_2 <- cnv_2[names(cnv_2) %in% our.names == FALSE] # exclude samples from our own cohort
cnv_2

$SM006
An object of class Seurat 
36498 features across 5805 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$SM011
An object of class Seurat 
36498 features across 5453 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$SM012
An object of class Seurat 
36498 features across 6890 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$SM017
An object of class Seurat 
36498 features across 8116 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$SM018
An object of class Seurat 
36498 features across 2970 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$G1003
An object of class Seurat 
36498 features across 9376 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$G523
An object of class Seurat 
36498 features across 5558 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$G549
A

In [ ]:
gbm_1 <- merge(cnv_1[[1]], y = unlist(cnv_1[2:length(cnv_1)]), project = "GBM", merge.data = T)
gbm_2 <- merge(cnv_2[[1]], y = unlist(cnv_2[2:length(cnv_2)]), project = "GBM", merge.data = T)
gbm <- merge(gbm_1, y = gbm_2, project = "GBM", merge.data = T)

In [ ]:
gbm

An object of class Seurat 
36498 features across 279753 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

In [ ]:
gbm_all <- readRDS('data/core_GBmap_seuratobj_list.rds')
gbm_all

$Yuan2018
An object of class Seurat 
28951 features across 12202 samples within 1 assay 
Active assay: RNA (28951 features, 0 variable features)

$Neftel2019_10x
An object of class Seurat 
22966 features across 13288 samples within 1 assay 
Active assay: RNA (22966 features, 0 variable features)

$Neftel2019_smart
An object of class Seurat 
21718 features across 5742 samples within 1 assay 
Active assay: RNA (21718 features, 0 variable features)

$Wang2019
An object of class Seurat 
38129 features across 23531 samples within 1 assay 
Active assay: RNA (38129 features, 0 variable features)

$Wang2020
An object of class Seurat 
17566 features across 12437 samples within 1 assay 
Active assay: RNA (17566 features, 0 variable features)

$Zhao2020
An object of class Seurat 
18774 features across 5182 samples within 1 assay 
Active assay: RNA (18774 features, 0 variable features)

$Bhaduri2020
An object of class Seurat 
22183 features across 14934 samples within 1 assay 
Active assay: RNA (2

In [ ]:
immune <- gbm_all[names(gbm_all) %in% c('Sankowski2019','Goswami2019','Pombo2021','Mathewson2021')] 
immune

$Sankowski2019
An object of class Seurat 
14398 features across 2139 samples within 1 assay 
Active assay: RNA (14398 features, 0 variable features)

$Goswami2019
An object of class Seurat 
14083 features across 5997 samples within 1 assay 
Active assay: RNA (14083 features, 0 variable features)

$Pombo2021
An object of class Seurat 
18370 features across 79981 samples within 1 assay 
Active assay: RNA (18370 features, 0 variable features)

$Mathewson2021
An object of class Seurat 
16270 features across 24310 samples within 1 assay 
Active assay: RNA (16270 features, 0 variable features)


In [ ]:
# annotate all immune cells as diploid cells
immune <- merge(immune[[1]], y = unlist(immune[2:length(immune)]), project = "GBM", merge.data = T)
immune$predCNA <- 'diploid'

In [ ]:
our.merged <- merge(our[[1]], y = unlist(our[2:length(our)]), project = "GBM", merge.data = T)
our.merged

An object of class Seurat 
36498 features across 39355 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

In [ ]:
gbm <- merge(gbm_1, y = c(gbm_2, immune, our.merged), project = "GBM", merge.data = T)
gbm

An object of class Seurat 
37559 features across 431535 samples within 1 assay 
Active assay: RNA (37559 features, 0 variable features)

In [ ]:
cnv <- data.frame(cnv_full_geneset = gbm@meta.data$predCNA)
rownames(cnv) <- colnames(gbm)

In [ ]:
write.table(cnv,
            'data/core_GBmap_cnv_metadata.csv',
           row.names = TRUE, col.names = TRUE, quote = FALSE, sep = ',')